# 🎬 LSTM for IMDb Movie Review Sentiment Analysis

Welcome to this 2-hour session on using Long Short-Term Memory (LSTM) networks for sentiment analysis! We'll build a model to predict whether a movie review is positive or negative. 

### 📘 Learning Objectives:

1.  **Understand Sentiment Analysis:** Learn what it is and why it's important.
2.  **Prepare Text Data:** Preprocess the IMDb movie review dataset for our AI model.
3.  **Build an LSTM Model:** Create a neural network using LSTMs, a special type of Recurrent Neural Network (RNN).
4.  **Train & Evaluate:** Train our model and see how well it performs.
5.  **Test with Your Own Reviews:** Predict the sentiment of any review you type!
6.  **Why LSTMs are better than RNNs for text:** Understand the improvement LSTMs offer for handling long sentences.

## 💡 Topic 1: What is Sentiment Analysis?

Sentiment analysis is the process of determining the emotional tone behind a series of words. In simple terms, it's figuring out if a piece of text is positive, negative, or neutral. 

For example:
-   `"This movie was fantastic!"` -> **Positive**
-   `"I did not enjoy the film at all."` -> **Negative**

We'll be using the famous IMDb dataset, which contains 50,000 movie reviews labeled as either positive or negative.

## ⚙️ Topic 2: Setting Up Our Environment

First, let's import the necessary libraries from TensorFlow and Keras. These tools help us build and train our AI model.

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 📦 Topic 3: Loading and Preparing the Data

We will load the IMDb dataset. The dataset comes pre-processed where words are already converted into numbers (integers). We'll limit our vocabulary to the top 10,000 most frequent words.

Then, we need to make sure all our reviews are the same length. We use `pad_sequences` to make every review 200 words long. Shorter reviews get padded with zeros, and longer ones are truncated.

In [2]:
# Parameters
vocab_size = 10000  # Number of words to consider as features
maxlen = 200       # Max length of reviews (in words)

# Load the data
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

print(f"Training examples: {len(x_train)}")
print(f"Test examples: {len(x_test)}")

# --- Padding Sequences ---
# Make all sequences (reviews) have the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

print(f"\nShape of padded training data: {x_train.shape}")
print(f"Shape of padded test data: {x_test.shape}")

Training examples: 25000
Test examples: 25000

Shape of padded training data: (25000, 200)
Shape of padded test data: (25000, 200)


### 🎯 Practice Task: Check the Data

Let's inspect one of the processed reviews to see what it looks like. It's just a list of numbers!

In [3]:
# Print the first review from the training set
print("First review (as numbers):")
print(x_train[0])

# The label '1' means positive, '0' means negative
print("\nLabel of first review:", y_train[0]) 

First review (as numbers):
[   5   25  100   43  838  112   50  670    2    9   35  480  284    5
  150    4  172  112  167    2  336  385   39    4  172 4536 1111   17
  546   38   13  447    4  192   50   16    6  147 2025   19   14   22
    4 1920 4613  469    4   22   71   87   12   16   43  530   38   76
   15   13 1247    4   22   17  515   17   12   16  626   18    2    5
   62  386   12    8  316    8  106    5    4 2223 5244   16  480   66
 3785   33    4  130   12   16   38  619    5   25  124   51   36  135
   48   25 1415   33    6   22   12  215   28   77   52    5   14  407
   16   82    2    8    4  107  117 5952   15  256    4    2    7 3766
    5  723   36   71   43  530  476   26  400  317   46    7    4    2
 1029   13  104   88    4  381   15  297   98   32 2071   56   26  141
    6  194 7486   18    4  226   22   21  134  476   26  480    5  144
   30 5535   18   51   36   28  224   92   25  104    4  226   65   16
   38 1334   88   12   16  283    5   16 4472  113

## 🧠 Topic 4: Why LSTMs are Better than Simple RNNs

A simple Recurrent Neural Network (RNN) has a major problem: the **vanishing gradient problem**. 

Imagine you're reading a very long review:

`"I went to see this film last week with my friends, and while the popcorn was a bit stale, the cinematography reminded me of the classic movies from the 1960s, which I adore, but the plot was so confusing and poorly written that by the end, I have to say it was a **terrible** experience."`

A simple RNN might forget the important early parts of the sentence (like `"...popcorn was stale..."` or `"...cinematography reminded me of classic movies..."`) by the time it reaches the end word (`"terrible"`). It struggles to maintain context over long sequences.

**LSTMs solve this!** They have a special memory cell and gates (input, output, forget gates) that allow them to remember or forget information selectively. This makes them excellent at understanding the context in long sentences, leading to much better performance on tasks like sentiment analysis of detailed reviews.

## 🤖 Topic 5: Building the LSTM Model

Now for the exciting part! We'll build our neural network layer by layer using Keras `Sequential` API.

1.  **Embedding Layer:** This turns the integer numbers (representing words) into dense vectors of a fixed size. It helps the model learn the relationships between words.
2.  **LSTM Layer:** This is the core of our model. We use a `Bidirectional` wrapper, which means it reads the review from start-to-end and end-to-start, capturing even more context!
3.  **Dense Layer:** A final, fully connected layer that outputs a single value, which will tell us if the review is positive or negative.

In [4]:
# Create the model
model = Sequential([
    # 1. Embedding Layer: Turns word indices into dense vectors of 128 dimensions
    Embedding(vocab_size, 128, input_length=maxlen),
    
    # 2. LSTM Layer: Processes the sequence of vectors. Bidirectional helps learn from both directions.
    Bidirectional(LSTM(64, return_sequences=False)), # return_sequences=False as we need output only at the end
    
    # 3. Dense Layer: The final output layer with a sigmoid activation for binary classification (0 or 1)
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


## 🏋️‍♀️ Topic 6: Training the Model

Now, we feed our training data (`x_train`, `y_train`) to the model. The model will learn to associate sequences of words with their sentiment (positive/negative).

This step will take a few minutes. Watch the `accuracy` and `val_accuracy` metrics increase!

In [5]:
print("Training the model... this might take a few minutes.")

# We'll use a portion of training data for validation during training
history = model.fit(x_train, y_train,
                    epochs=3, # We'll train for 3 full cycles
                    batch_size=128, # Process 128 reviews at a time
                    validation_split=0.2) # Use 20% of training data for validation

print("\n✅ Model training complete!")

Training the model... this might take a few minutes.
Epoch 1/3
157/157 [==============================] - 67s 392ms/step - loss: 0.4694 - accuracy: 0.7596 - val_loss: 0.3176 - val_accuracy: 0.8686
Epoch 2/3
157/157 [==============================] - 60s 385ms/step - loss: 0.2400 - accuracy: 0.9080 - val_loss: 0.3057 - val_accuracy: 0.8774
Epoch 3/3
157/157 [==============================] - 60s 384ms/step - loss: 0.1638 - accuracy: 0.9406 - val_loss: 0.3403 - val_accuracy: 0.8732

✅ Model training complete!


## 📈 Topic 7: Evaluating the Model

Let's see how well our trained model performs on the test data it has never seen before.

In [6]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)

print(f"\nTest Accuracy: {accuracy*100:.2f}%")
print(f"Test Loss: {loss:.4f}")

782/782 [==============================] - 48s 61ms/step - loss: 0.3687 - accuracy: 0.8602

Test Accuracy: 86.02%
Test Loss: 0.3687


## 🧪 Topic 8: Testing with Your Own Reviews!

This is the most fun part. Let's write a function that takes your review, processes it, and predicts the sentiment using our trained model.

We need to convert your text into the same numerical format the model was trained on.

In [7]:
import numpy as np

# Get the word-to-index mapping from the IMDb dataset
word_index = imdb.get_word_index()

# Create a reverse word index (index -> word)
reverse_word_index = {v: k for k, v in word_index.items()}

# The first few indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # UNK for unknown words

def predict_sentiment(review_text):
    """Preprocesses a user's review and predicts its sentiment."""
    # Convert review to lowercase and split into words
    words = tf.keras.preprocessing.text.text_to_word_sequence(review_text)
    
    # Convert words to their integer indices
    encoded_review = [word_index.get(word, 2) for word in words] # Use 2 for unknown words
    
    # Pad the sequence to the same length (200)
    padded_review = pad_sequences([encoded_review], maxlen=maxlen)
    
    # Make a prediction
    prediction = model.predict(padded_review)
    
    # Interpret the prediction
    if prediction[0][0] > 0.5:
        print(f"Prediction Score: {prediction[0][0]:.4f} -> 😊 Positive Review")
    else:
        print(f"Prediction Score: {prediction[0][0]:.4f} -> 😞 Negative Review")

### 🎯 Practice Task: Write Your Own Reviews!

Now it's your turn! Try different reviews in the cell below and see what the model predicts. Test it with clearly positive, clearly negative, and even more complex, longer reviews.

In [9]:
# --- Positive Review Example ---
my_review_1 = "This film was absolutely brilliant. The acting was superb and the storyline was gripping from start to finish. I would recommend it to everyone."
print("Testing a positive review:")
predict_sentiment(my_review_1)

print("-"*50)

# --- Negative Review Example ---
my_review_2 = "I was so disappointed with this movie. The plot was incredibly boring and the characters were not believable at all. I almost walked out of the theater."
print("Testing a negative review:")
predict_sentiment(my_review_2)

print("-"*50)

print("\n🧪 Now, write your own review inside the quotes and run the cell!")
your_own_review = "Replace this text with your movie review."
predict_sentiment(your_own_review)

Testing a positive review:
1/1 [==============================] - 0s 70ms/step
Prediction Score: 0.8839 -> 😊 Positive Review
--------------------------------------------------
Testing a negative review:
1/1 [==============================] - 0s 52ms/step
Prediction Score: 0.0114 -> 😞 Negative Review
--------------------------------------------------

🧪 Now, write your own review inside the quotes and run the cell!
1/1 [==============================] - 0s 51ms/step
Prediction Score: 0.4573 -> 😞 Negative Review


## 🎉 Final Revision Assignment

Congratulations on building your first sentiment analysis model with LSTMs! To practice and reinforce what you've learned, here are a few tasks you can try at home.

**Task 1: The Sarcastic Review**
-   Write a sarcastic review like `"Wow, another two hours of my life I'll never get back. Truly a masterpiece of boring cinema."` and see if the model can correctly classify it as negative.

**Task 2: The Mixed Feelings Review**
-   Write a review with both positive and negative points, like `"The special effects were incredible, but the story was weak and predictable."` See which way the model leans.

**Task 3: Change Model Parameters**
-   Go back to the model building cell (`Topic 5`). Try changing the number of LSTM units from `64` to `32` or `128`. Re-train the model and see how the accuracy changes.

**Task 4: Change `maxlen`**
-   In `Topic 3`, change the `maxlen` parameter from `200` to `300`. How does this affect training time and accuracy? Do you think it would help or hurt the model's performance?

**Task 5: Explore the Vocabulary**
-   Write a short script to see what word corresponds to a specific number. For example, what word is represented by the number `50`? (Hint: Use the `reverse_word_index` we created).

**Task 6 (Advanced): Try a Different Optimizer**
-   In the `model.compile()` step, change the optimizer from `'adam'` to `'rmsprop'`. Re-train and compare the results.